In [2]:
%load_ext autoreload
%autoreload 2

In [82]:
import urban_dictionary_scraper
import logging
import pickle
from scipy import stats
import pandas as pd
from tqdm.notebook import tqdm
from collections import OrderedDict
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
import itertools
import numpy as np
import re
import random

In [2]:
logging.basicConfig(level=logging.INFO)
session = urban_dictionary.get_session(throttle=0.1, expiry = (7*24*3600))

In [ ]:
"""
all_urls = urban_dictionary_scraper.fetch_all_word_urls(session)
with open("all_urls.pickle", "wb") as f:
    pickle.dump(all_urls, f, pickle.HIGHEST_PROTOCOL)
"""

In [5]:

with open("all_urls.pickle", "rb") as f:
    to_fetch = pickle.load(f)
    
with open("all_words.pickle", "rb") as f:
    already_done = pickle.load(f)
    for key in already_done.keys():
        del to_fetch[key]
        
done = 100 * len(already_done) / (len(already_done) + len(to_fetch))
print(f"Done {done:.2f} percent")

FileNotFoundError: [Errno 2] No such file or directory: 'all_urls.pickle'

In [ ]:
t = ThreadPool(5)
#with ThreadPoolExecutor(max_workers=5) as executor:
try:
    fetch_all_definitions(session, to_fetch, already_done, save_interval=10000, executor=t)    
finally:
    t.terminate()
    t.join()

In [6]:


with open("data/all_words.pickle", "rb") as f:
    words = pickle.load(f)
    
    

In [84]:
def is_clean(word, upvote_smoothing=20, min_upvote_percentage=0.2, max_word_length=40, max_symbols=2):
    smoothed_upvotes = word.upvotes / (word.upvotes + word.downvotes + upvote_smoothing)
    if smoothed_upvotes < min_upvote_percentage:
        return False
    elif len(word.word) > max_word_length:
        return False
    elif len(re.findall(r"[^\w .]", word.word)) > max_symbols:
        return False
    else:
        return True
    
clean_list = [
    (k, urban_dictionary_scraper.UrbanDictionaryWord(
        title=e.title,
        url=e.url,
        definitions=e.definitions[:1],
    ))
    for k,e in words.items() if is_clean(e.definitions[0])
]
random.shuffle(clean_list)
cleaned_words = OrderedDict(clean_list)

print(f"Words reduced by {len(cleaned_words) / len(words)}")

with open("data/cleaned_words_top_def_mu02_randomized.pickle", "wb") as f:
    pickle.dump(cleaned_words, f, pickle.HIGHEST_PROTOCOL)

Words reduced by 0.42859665990996765


In [8]:
defns = pd.DataFrame(
    [
        [e.word, e.meaning, e.examples[0], e.creation_epoch, e.upvotes, e.downvotes]
        for e in itertools.chain.from_iterable(e.definitions for e in words.values())
    ],
    columns=["word", "meaning", "example", "creation_epoch", "upvotes", "downvotes"]
)

In [26]:
smoothing_prior = 20
defns["smoothed_upvotes"] = defns["upvotes"] / (defns["upvotes"] + defns["downvotes"] + smoothing_prior)

In [27]:
defns["smoothed_upvotes"].quantile(np.linspace(0.1, 1, 10))

0.1    0.000000
0.2    0.045455
0.3    0.086957
0.4    0.129032
0.5    0.178571
0.6    0.241935
0.7    0.325000
0.8    0.440678
0.9    0.593407
1.0    0.999809
Name: smoothed_upvotes, dtype: float64

In [85]:
cleaned_defs = defns[:]
cleaned_defs = cleaned_defs[cleaned_defs["smoothed_upvotes"] >= 0.2]
cleaned_defs = cleaned_defs[cleaned_defs.word.str.len() <= 40]
cleaned_defs = cleaned_defs[cleaned_defs.word.str.count("[^\w .]") <= 2]
print(f"Reduction from {len(defns)} to {len(cleaned_defs)} ({len(cleaned_defs) / len(defns)})")

Reduction from 2961824 to 1389767 (0.4692267332562637)


In [45]:
defns.word.str.count("[^\w ].").describe()

count    2.961824e+06
mean     7.649577e-02
std      4.030062e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      9.800000e+01
Name: word, dtype: float64

,word,meaning,example,creation_epoch,upvotes,downvotes,smoothed_upvotes
31555,AHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH,AHHHHHHHHHHHHHHHH is a popular phrase which ca...,Bloke 1: Jim Bob the car isn't working!\nBloke...,1.386115e+09,27,6,0.509434
2689686,tsavo,"Tsavo, a progressive rock band from the Seattl...",Person 1: Dude have you ever heard TSAVO?\nPer...,1.196122e+09,49,10,0.620253
824258,Evian,French water product in the Alps. The brand is...,Evian is the world leader in terms of sales on...,1.106438e+09,57,32,0.522936
2564875,Testicle Board,"A flat piece of plastic, wood, or other non-he...","Yo man, pass me the testicle board, my boys ar...",1.108685e+09,41,9,0.585714
521786,compassion,The only true path to wisdom.,All wisdom and compassion are eternaly linked.,1.150848e+09,261,73,0.737288
2333909,skinned my dink,"To screw some bitch all night, real hard so th...","Oh man, I really I skinned my dink on that bit...",1.144282e+09,33,5,0.568966
641510,degeneration-x,The Degeneration-X (DX) are the most controver...,The Degeneration-X have made their mark in WWE...,1.150675e+09,44,22,0.511628
331811,brownie,"something thats very cool, hip, or awesome. We...","Man, niagra falls is so brownie.\n\nor\n\nWow ...",1.255219e+09,344,233,0.576214
1067177,gooche,The space between ones genitals and ones poo pot.,"This one time, i wiped my ass the wrong way an...",1.065139e+09,66,44,0.507692
101434,artery fuck,Food so inherently fatty that your arteries ar...,"""scones and clotted cream, I can't get enough ...",1.254960e+09,41,8,0.594203


In [67]:
defns[defns.word.str.len() > 40].sample(n=20)

,word,meaning,example,creation_epoch,upvotes,downvotes,smoothed_upvotes
2597220,The Past is Never Perfect but the Future Has a...,We can only examine events in the past which n...,I used to predict the future back in 1987 but ...,1.533946e+09,0,0,0.000000
2826663,"Whatever floats your boat, tickles your pickle...",The phrase used to confirm something when you ...,Person 1: So Im gonna ask Grey to marry me tod...,1.356912e+09,4,1,0.160000
2532471,take a butterfly knife to your tender parts,what the father of a teenage girl does to you ...,"Angry War Veteran Father: ""If you hurt her, I ...",1.237766e+09,9,17,0.195652
511023,cognitive marijuanal grudge reduction therapy,Therapy referring to the process of blazing wi...,Man I haven't talked to my sister for 7 years ...,1.218931e+09,6,2,0.214286
1245536,"If Jesus can walk on water, can he swim on land",I stole this from Bo burnham,Bo burnham: I’m so hashtag deep\nMe: what a li...,1.549498e+09,15,0,0.428571
1282722,it rubs the lotion on its skin or else it gets...,that creepy thing buffalo bill would say in th...,guy 1: hey phil it rubs the lotion on its skin...,1.215648e+09,116,82,0.532110
2960021,8 Simple Rules For Dating My Teenage Daughter,An American sitcom starring John Ritter most p...,\n8 Simple Rules For Dating My Teenage Daught...,1.144886e+09,47,25,0.510870
2937765,"!""£$%^&*()_+qwertyuiop{}asdfghjkl:@~|zxcvbnm<>?",When you are sitting in school trying to be a ...,"...I is bored I feel, !""£$%^&*()_+QWERTYUIOP{}...",1.570666e+09,5,1,0.192308
2574348,thats how I role crazy and out of control,To make it seem like you are crazier then you ...,"that was dumb adam, yea thats how I role crazy...",1.216858e+09,9,2,0.290323
712798,Don't Hate The Playa/Playette Hate The Game,Used by pimps and pimpettes to clear up the tr...,When Gina the pimpette moved on to other guys ...,1.045699e+09,98,285,0.243176


In [46]:
defns[defns.word.str.count("[^\w .]") > 2].sample(n=20)

,word,meaning,example,creation_epoch,upvotes,downvotes,smoothed_upvotes
1174348,HIM (His-Infernal-majesty),"a truly amazing band, need i say more","""dude i saw HIM live last nite they fucking RO...",1.110154e+09,93,57,0.547059
2049016,Q-W-E-R D-F B,The basic League of Legends hotkeys.\rQ-W-E-R ...,Player 1: Hey what's your LoL hotkeys?\nPlayer...,1.353974e+09,26,11,0.456140
1540267,--Luke Skywalker--,"Luke becomes a Jedi Knight, almost shacks up w...","Peep 1: That guy is cool\nPeep 2: Yeah, he was...",1.158192e+09,46,37,0.446602
2525418,T///T,Based on the T_T face; T///T means a crying fa...,Oh no! I forgot to watch Inuyasha today. T///T,1.233274e+09,6,0,0.230769
1125242,Half-racist/Half-Racism,has friends of a different races but won't dat...,A white person has black friends but won't dat...,1.459728e+09,0,0,0.000000
2246478,"sex """"positions omg""""",Somone who gets excited on teh thought of sexu...,"omg sex """"positions omg"""" are so cool",1.201565e+09,170,443,0.268562
712400,"Don't be a ""Todd.""",What exactly is a Todd? A Todd is a hyper-sexu...,"Don't be a ""Todd.""",1.554077e+09,0,1,0.000000
2942228,(づ ◕‿‿◕ )づ,Amazing,That lightly buttered crumpet with a little bi...,1.564877e+09,2,1,0.086957
2941716,(>-),A peace sign. A meaning of truce.,(>-) <3 (:,1.298506e+09,1,1,0.045455
2836703,Whomst’d’ve’ly’yaint’nt’ed’ies’s’y’es’ll,The superior form of whomst’dve.\rWith this ne...,Guy 1: Whomst’d’ve’ly’yaint’nt’ed’ies’s’y’es’l...,1.529194e+09,32,0,0.615385


In [65]:
(defns["meaning"].str.len() + defns["example"].str.len()).quantile(np.linspace(0.01, 1, 100))

0.01       38.00
0.02       47.00
0.03       54.00
0.04       59.00
0.05       63.00
          ...   
0.96      743.00
0.97      833.00
0.98      977.00
0.99     1266.51
1.00    91521.00
Length: 100, dtype: float64

In [61]:
lng_defs = defns[defns["meaning"].str.len() > 985]
(lng_defs["upvotes"] + lng_defs["downvotes"]).describe()

count     29628.000000
mean        257.874713
std        1731.727996
min           0.000000
25%          10.000000
50%          44.000000
75%         162.000000
max      230221.000000
dtype: float64

In [62]:
lng_defs = defns[defns["meaning"].str.len() < 985]
(lng_defs["upvotes"] + lng_defs["downvotes"]).describe()

count    2.940324e+06
mean     7.464941e+01
std      8.058769e+02
min      0.000000e+00
25%      2.000000e+00
50%      8.000000e+00
75%      3.100000e+01
max      6.271090e+05
dtype: float64